In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import KFold, cross_val_score
import xgboost, math
%matplotlib inline

/home/jonathan/.local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df = pd.read_csv('train_revised.csv', parse_dates=['travel_date'], dayfirst=True)

In [5]:
train = df.groupby(['ride_id', 'travel_date', 'travel_time', 'travel_from', 'max_capacity']).size().reset_index(name='Count') #sort=False if needed?

In [4]:
sample = pd.read_csv('sample_submission.csv')

In [6]:
# Time in minutes
train["travel_time"] = train["travel_time"].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))

In [7]:
train['day'] = train['travel_date'].dt.dayofweek

In [8]:
test = pd.read_csv('test_questions.csv', parse_dates=['travel_date'], dayfirst=True).drop(['car_type', 'travel_to'], axis=1)
test["travel_time"] = test["travel_time"].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
test['day'] = test['travel_date'].dt.dayofweek

In [11]:
train['t'] = 0
test['t'] = 1

In [20]:
X = pd.concat([train, test], sort=False)

In [21]:
Xd = pd.get_dummies(X, columns=['travel_from', 'day'])

In [22]:
Xd.head()

,ride_id,travel_date,travel_time,max_capacity,Count,t,travel_from_Awendo,travel_from_Homa Bay,travel_from_Kehancha,travel_from_Kendu Bay,...,travel_from_Rongo,travel_from_Sirare,travel_from_Sori,day_0,day_1,day_2,day_3,day_4,day_5,day_6
0,1442,2017-10-17,435,49,1.0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,5437,2017-11-19,432,49,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,5710,2017-11-26,425,49,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,5777,2017-11-27,430,49,5.0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4,5778,2017-11-27,432,49,31.0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [24]:
X_train = Xd.loc[Xd['t'] == 0].drop(['Count', 'ride_id', 'travel_date'], axis=1)
y_train = Xd.loc[Xd['t'] == 0]['Count']
regr = RandomForestRegressor(n_estimators=200, criterion="mae", max_depth=10, n_jobs=-1) 
regr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [26]:
print(mean_absolute_error(regr.predict(X_train), y_train))

3.3197751640262445


In [27]:
X_test = Xd.loc[Xd['t'] == 1].drop(['Count', 'ride_id', 'travel_date'], axis=1)
pred = regr.predict(X_test)

In [31]:
sample['number_of_ticket'][5:] = pred[5:]
sample.head(10)

/home/jonathan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,ride_id,number_of_ticket
0,247,4.000
1,256,2.000
2,275,5.000
3,285,7.000
4,286,9.000
5,287,3.995
6,288,9.860
7,292,8.565
8,298,9.830
9,302,9.820


In [33]:
sample.to_csv('rfr_dummy_test.csv', index=False)